# 1- import libraries 

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

# 2-  Calculate Entropy

In [2]:
def entropy(y):
    class_counts = np.bincount(y)
    probabilities = class_counts / len(y)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

# 3- Calculate Information Gain (IG)

In [3]:
def information_gain(X_column, y, threshold):
    # Split the data into two parts
    left_mask = X_column <= threshold
    right_mask = X_column > threshold
    
    # Get left and right splits
    left_y, right_y = y[left_mask], y[right_mask]
    
    # Calculate entropy before and after split
    before_split_entropy = entropy(y)
    n = len(y)
    left_entropy = entropy(left_y)
    right_entropy = entropy(right_y)
    after_split_entropy = (len(left_y) / n) * left_entropy + (len(right_y) / n) * right_entropy
    
    # Information gain is the reduction in entropy
    return before_split_entropy - after_split_entropy

# 4- Node Class

In [4]:
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature  # Index of the feature
        self.threshold = threshold  # Value of threshold for split
        self.left = left  # Left subtree
        self.right = right  # Right subtree
        self.value = value  # For leaf node, this will store the class

# 5-  Decision Tree Class

In [5]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    # Build the tree recursively
    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    # Recursively build tree
    def _build_tree(self, X, y, depth = 0):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))
        
        # Stopping criteria
        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return DecisionNode(value = leaf_value)

        # Greedily select the best split
        best_feature , best_threshold = self._best_criteria(X, y)
        
        # Create the child nodes (subtrees)
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left_subtree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return DecisionNode(feature = best_feature, threshold = best_threshold, left = left_subtree, right = right_subtree)

    # Find the best feature and threshold to split
    def _best_criteria(self, X, y):
        best_gain = -1
        best_feature, best_threshold = None, None
        
        for feature in range(X.shape[1]):
            X_column = X[:, feature]
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                gain = information_gain(X_column, y, threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
                    
        return best_feature, best_threshold

    # Split data based on threshold
    def _split(self, X_column, threshold):
        left_idxs = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()
        return left_idxs, right_idxs

    # Get the most common label
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    # Predict function for single sample
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    # Predict function for multiple samples
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])


# 6- Example 

In [6]:
if __name__ == "__main__":
    # Example dataset
    data = {'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
            'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
            'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
            'Windy': ['False', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'True'],
            'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']}
    
    df = pd.DataFrame(data)
    
    # Encoding categorical data
    df['Outlook'] = df['Outlook'].map({'Sunny': 0, 'Overcast': 1, 'Rain': 2})
    df['Temperature'] = df['Temperature'].map({'Hot': 0, 'Mild': 1, 'Cool': 2})
    df['Humidity'] = df['Humidity'].map({'High': 0, 'Normal': 1})
    df['Windy'] = df['Windy'].map({'False': 0, 'True': 1})
    df['Play'] = df['Play'].map({'No': 0, 'Yes': 1})
    
    X = df.drop(columns='Play').values
    y = df['Play'].values
    
    # Initialize and fit decision tree
    tree = DecisionTree(max_depth=3)
    tree.fit(X, y)
    
    # Make predictions
    predictions = tree.predict(X)
    print("Predictions:", predictions)

Predictions: [0 0 1 1 1 0 1 0 1 1 1 1 1 1]
